# Exploring and clustering neighborhoods in Toronto
Coursera capstone course for applied data science

1. scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
To create the above dataframe:
  * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
  * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
  * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
  * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
  * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
  * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [61]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.display import display_html

Here we load the table using requests and BeautifulSoup.  
The first table with the class "wikitable" is the one we are looking for.  
An interesting Beautiful Soup tutorial can be found here https://www.youtube.com/watch?v=ng2o98k983k

In [47]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table', class_='wikitable')

Now, we load the dataframe using the read_html method from the IPython library.    
Next lines inspired from this tutorial 
https://beenje.github.io/blog/posts/parsing-html-tables-in-python-with-pandas/

In [92]:
# render the table from the HTML string
# display_html(str(table), raw=True)
# load the table in a dataframe
df = pd.read_html(str(table),header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Table cleanup

In [94]:
# rename the columns
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

# delete the rows where the Borough is not assigned
df = df[df.Borough != 'Not assigned']

# group by Borough and combine the Neighborhood
# see here https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
# https://stackoverflow.com/questions/35068722/pandas-assign-value-of-one-column-based-on-another
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally we print the dimensions of the table

In [95]:
df.shape

(103, 3)